<div align="center"><a href="https://colab.research.google.com/github/deepkapha/EarthScanWebinar/blob/main/notebook/W2W_dataset_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></div>

In [ ]:
import sys 
sys.path.append('..')

# 1. Import the necessaary libraries

In [ ]:
import os
import csv
import gdown
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.colors as mcolors
# from PIL import Image
from os.path import join as pjoin
from utils.well_log_plots import log_plots
from sklearn.preprocessing import LabelEncoder

In [ ]:
pd.set_option('display.max_columns', 500)
overlap = {name for name in mcolors.CSS4_COLORS
           if f'xkcd:{name}' in mcolors.XKCD_COLORS}

# 2. Download the dataset from publicly avalialable google drive link

In [ ]:
train_data_url = 'https://drive.google.com/file/d/1TkRT5TeX7slPDL20OqYqFLhGX5iK4RHc/view?usp=share_link'
test_data_url = 'https://drive.google.com/file/d/1ooDiCKweUrduyy0Q1P7KIXnl04BQqH0_/view?usp=share_link'

data_root = '../raw_data'
os.makedirs(data_root, exist_ok = True)
train_path = pjoin(data_root, "train.csv")
test_path = pjoin(data_root, "test.csv")

In [ ]:
if not os.path.isfile(train_path):
    gdown.download(url=train_data_url, output=train_path, quiet=False, fuzzy=True)

if not os.path.isfile(train_path):
    gdown.download(url=test_data_url, output=test_path, quiet=False, fuzzy=True)

# 3. 

In [ ]:
start_well=30
end_well=98

In [ ]:
well_train = load_data(train_path)
well_test = load_data(test_path)

In [ ]:
print("Shape of Well data:\n\tTraining Well: {}\n\tTesting Well: {}".format(well_train.shape, well_test.shape))

# Description of dataset

The provided dataset contains well logs, interpreted lithofacies and lithostratigraphy for 90+ released wells from offshore Norway. The well logs include the well name (WELL), the measured depth, x,y,z location for the wireline measurement as well as the well logs CALI, RDEP, RHOB, DHRO, SGR,  GR, RMED, RMIC, NPHI, PEF, RSHA, DTC, SP, BS, ROP, DTS, DCAL, MUDWEIGHT. An explanation of the abbreviations is shown in the figure below. 

<center>
<img src="../asset/well_log_abbreviations.png">
</center>

Click [here](https://drive.google.com/drive/folders/1GIkjq4fwgwbiqVQxYwoJnOJWVobZ91pL) to check the results.

Given below is the Statistical description of the few important logs from the raw_data


| Log Name | min_scale | max_scale | description |
|----------|-----------|-----------|-------------|
| CALI     |  6        |   24      |             |
| GR       |  0        |   150     |             |
| SP       |  -150     |   150     |             |
| SGR      |  0        |   150     |             |
| RSHA     |  0.2      |   200     |             |
| RMED     |  0.2      |   200     |             |
| RDEP     |  0.2      |   200     |             |
| RMIC     |  0.2      | 200 ohm-m |             |
| RXO      |  0.2      | 200 ohm-m |             |                 
| ROPA     |   0       |    50     |             |
| ROP      |   0       |    50     |             |
| DTC      |   40       |  240      |             |
| DTS      |   40       |  240      |             |
| NPHI     |   0.05    |  -0.15    |             |
| RHOB     |   0.95    |   2.95    |             |
| PEF      |  0        |10 barns/electron|       |             
| DCAL     |  6        |    24     |             |
| DRHO     |  -0.2     |   1       |             |
|MUDWEIGHT |    0       |  150         |             |
|BS        |     6      |    24       |             |

In [ ]:
total_training_wells = count_well(well_train)
total_testing_wells = count_well(well_test)

print('Total number of wells present in the dataset:\n\tTraining Well: {}\n\tTesting Well: {}'.format(total_training_wells, total_testing_wells))

In [ ]:
print(f"Unique wells present in the dataset are:\n\t{well_train.WELL.unique()}")

In [ ]:
well_train_names = get_well_names(well_train)
well_test_names = get_well_names(well_test)

In [ ]:
overlapped_well_count, overlapped_well_name = get_overlapping_well(well_train, well_test)

In [ ]:
well_train.head()

In [ ]:
print("Max mud weight value is",np.max(well_train["MUDWEIGHT"]))
print("Max mud weight value is",np.min(well_train["MUDWEIGHT"]))

In [ ]:
well_test.head()

In [ ]:
get_random_well(well_train, 2020).head()

In [ ]:
print('Displaying %age of missing data points for all columns in training data\n')
print(percet_missing_data(well_train))
percet_missing_data(well_train).plot(kind='bar')
plt.show()
print('\n' + '='*80 + '\n\nDisplaying %age of missing data points for all columns in testing data\n')
print(percet_missing_data(well_test))
percet_missing_data(well_test).plot(kind='bar')
plt.show()

In [ ]:
#remove_column_with_half_of_nan_value(well_train)
#remove_column_with_half_of_nan_value(well_test)

In [ ]:
training_group_names = group_identification(well_train)
print()
testing_group_names = group_identification(well_test)

In [ ]:
def get_nonoverlapping_groups(train_group, test_group):

    count_nonoverlapping_groups = 0
    nonoverlapped_groups = []
    for group in test_group:
        if (group in train_group) == False:
            count_nonoverlapping_groups+=1
            nonoverlapped_groups.append(well)
        if count_nonoverlapping_groups !=0:
            print("Total numbers of GROUPS in testing wells that is not overlapping in training wells: {} and their names are {}".format(count_nonoverlapping_groups, nonoverlapped_groups))
            return (count_nonoverlapping_groups, nonoverlapped_groups)
        else:
            print("Total numbers of GROUPS in testing wells that is not overlapping in training wells: {}".format(count_nonoverlapping_groups))
            return (count_nonoverlapping_groups, None)

nonoverlapped_groups_count, nonoverlapped_groups_name = get_nonoverlapping_groups(training_group_names, testing_group_names)

In [ ]:
def missing_group_info(data):
    print('Total data points with missing Group information: {}'.format(data.GROUP.isna().sum()))

missing_group_info(well_train)
missing_group_info(well_test)

In [ ]:
def remove_formation_column(data):

    print('Removing FORMATION column from the dataset')
    data.drop(['FORMATION'], axis = 1, inplace = True)

In [ ]:
remove_formation_column(well_train)
remove_formation_column(well_test)

In [ ]:
well_train.head()

In [ ]:
well_test.head()

In [ ]:
get_random_well(well_train, 2020).head()

In [ ]:
def get_undeviated_well_info(data):
    well_names = []
    for well_name in get_well_names(data):
        well_data = data[data.WELL == well_name]
        if (well_data.X_LOC.value_counts().shape[0] == 1) and (well_data.Y_LOC.value_counts().shape[0] == 1):
            well_names.append(well_name)
            
    print('Total Number of undeviated wells present in the dataset is {} and their names are:\n{}'.format(len(well_names), 
                                                                                                          well_names))
    return well_names

In [ ]:
undeviated_wells = get_undeviated_well_info(well_train)

In [ ]:
def plot3Dlocation(rand_well):
    x_loc = rand_well.X_LOC
    y_loc = rand_well.Y_LOC
    z_loc = np.abs(rand_well.Z_LOC)

    fig = plt.figure(figsize=(12, 12))
    ax = plt.axes(projection='3d')
    ax.plot3D(x_loc, y_loc, np.abs(z_loc))

    ax.plot3D(x_loc.values[0], y_loc.values[0], z_loc.values[0], marker='s', color='black', ms=8)
    ax.plot3D(x_loc.values[-1], y_loc.values[-1], z_loc.values[-1], marker='*', color='red', ms=8)
    ax.set_xlabel('X Location')
    ax.set_ylabel('Y Location')
    ax.set_zlabel('TVD')
    ax.invert_zaxis()
    plt.show()

In [ ]:
def plot2Dlocation(rand_well):
    x_loc = rand_well.X_LOC
    y_loc = rand_well.Y_LOC
    z_loc = np.abs(rand_well.Z_LOC)

    _, ax = plt.subplots(1, 3, figsize = (20, 5))

    ax[0].plot(x_loc, y_loc)
    ax[0].set_title('X Location vs Y Location')
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_xlabel('X')
    ax[0].set_ylabel('Y')

    ax[1].plot(x_loc, z_loc)
    ax[1].set_title('X Location vs TVDss')
    ax[1].set_xticks([])
    ax[1].set_xlabel('X')
    ax[1].set_ylabel('TVDss')
    ax[1].plot(x_loc.values[0], z_loc.values[0], marker='s', color='black', ms=8)
    ax[1].plot(x_loc.values[-1], z_loc.values[-1], marker='*', color='red', ms=8)
    ax[2].invert_yaxis()

    ax[2].plot(y_loc, z_loc)
    ax[2].set_title('Y Location vs TVDss')
    ax[2].set_xticks([])
    ax[2].set_xlabel('Y')
    ax[2].set_ylabel('TVDss')
    ax[2].plot(y_loc.values[0], z_loc.values[0], marker='s', color='black', ms=8)
    ax[2].plot(y_loc.values[-1], z_loc.values[-1], marker='*', color='red', ms=8)
    ax[2].invert_yaxis()
    plt.show()

In [ ]:
rand_well = get_random_well(well_train)
plot2Dlocation(rand_well)
plot3Dlocation(rand_well)

In [ ]:
def get_well_with_no_group_info(data, total_num_wells, well_names):

    well_with_missing_group_info = []

    for well_names_index in range(total_num_wells):
        single_well_name = well_names[well_names_index]
        single_well_data = data[data['WELL'] == single_well_name]
        total_group_data_missing = single_well_data.GROUP.isna().sum()
        if total_group_data_missing != 0:
            print('Well "{0}" is having missing information on GROUP and total data points missing this information is {1}'.format(single_well_name, total_group_data_missing))
            well_with_missing_group_info.append(single_well_name)
    print('\nTotal {} Wells have missing group information!'.format(len(well_with_missing_group_info)))
    return well_with_missing_group_info

well_train_with_missing_group_info = get_well_with_no_group_info(well_train, total_training_wells, well_train_names)
well_test_with_missing_group_info = get_well_with_no_group_info(well_test, total_testing_wells, well_test_names)

In [ ]:
def fill_group_na_value(data, well_name, method = 'bfill'):
    return data[data['WELL'] == well_name].GROUP.fillna(method = method)

In [ ]:
well_train.GROUP.loc[well_train[well_train['WELL'] == well_train_with_missing_group_info[0]].GROUP.index] = fill_group_na_value(well_train, well_train_with_missing_group_info[0])
well_train.GROUP.loc[well_train[well_train['WELL'] == well_train_with_missing_group_info[1]].GROUP.index] = fill_group_na_value(well_train, well_train_with_missing_group_info[1])
well_train.GROUP.loc[well_train[well_train['WELL'] == well_train_with_missing_group_info[2]].GROUP.index] = fill_group_na_value(well_train, well_train_with_missing_group_info[2])
well_train.GROUP.loc[well_train[well_train['WELL'] == well_train_with_missing_group_info[3]].GROUP.index] = fill_group_na_value(well_train, well_train_with_missing_group_info[3])

In [ ]:
percet_missing_data(well_train)

In [ ]:
def xaxis(ax, xmin, xmax, spines_pos, color, log_name, name_size, y):
    ax.set_xticks((xmin, xmax))
    ax.spines['top'].set_position(('outward', spines_pos))
    ax.spines['top'].set_color(color)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.xaxis.set_label_coords(0.5, y)
    ax.set_xlabel(log_name, color = color, size = name_size)
    ax.tick_params(axis = 'x', colors = color, size = 0, labelsize = 10)

def log_plot(logs):
    _, ax = plt.subplots(1,5, figsize = (10, 15), sharey = True)
    ax[0].invert_yaxis()
    ax[0].grid()

    ax_gr_wrap = ax[0].twiny()
    ax_sp_wrap = ax[0].twiny()

    ax_rm_wrap = ax[1].twiny()
    ax_rd_wrap = ax[1].twiny()

    ax_nphi_wrap = ax[2].twiny()

    ax[0].plot(logs.CALI, list(range(logs.DEPTH_MD.shape[0])), 'b')
    ax_gr_wrap.plot(logs.GR, list(range(logs.DEPTH_MD.shape[0])), 'r')
    ax_sp_wrap.plot(logs.SP, list(range(logs.DEPTH_MD.shape[0])), 'g' )

    ax[1].plot(logs.RSHA, list(range(logs.DEPTH_MD.shape[0])), 'b')
    ax_rm_wrap.plot(logs.RMED, list(range(logs.DEPTH_MD.shape[0])), 'r')
    ax_rd_wrap.plot(logs.RDEP, list(range(logs.DEPTH_MD.shape[0])), 'g')

    ax[2].plot(logs.RHOB, list(range(logs.DEPTH_MD.shape[0])), 'b')
    ax_nphi_wrap.plot(logs.NPHI, list(range(logs.DEPTH_MD.shape[0])), 'r')

    ax[3].plot(logs.DTC, list(range(logs.DEPTH_MD.shape[0])), 'b')

    ax[0].set_xlim(6, 24)
    ax_gr_wrap.set_xlim(0, 150)
    ax_sp_wrap.set_xlim(-150, 150)

    ax[1].set_xlim(0.2, 200)
    ax_rm_wrap.set_xlim(0.2, 200)
    ax_rd_wrap.set_xlim(0.2, 200)

    ax[1].set_xscale('log')
    #ax_rm_wrap.set_xscale('log')
    #ax_rd_wrap.set_xscale('log')

    ax[2].set_xlim(0.95, 2.95)
    ax_nphi_wrap.set_xlim(-0.15, 1.05)
    ax_nphi_wrap.invert_xaxis()

    ax[3].set_xlim(40, 240)
    ax[3].invert_xaxis()

    logs_group_temp = logs.GROUP.fillna('N/A')
    encoder_temp = LabelEncoder()
    encoder_temp.fit(logs_group_temp)
    logs_group_temp_encoded = pd.Series(encoder_temp.transform(logs_group_temp))
    cl = np.repeat(np.expand_dims(logs_group_temp_encoded.values,1), 200, 1)
    ax[4].imshow(cl, interpolation='none', aspect='auto')


    ax[4].set_xticklabels([])

    xaxis(ax[0], 6, 24, 0, 'b', 'CALI', 10, 1.003)
    xaxis(ax_gr_wrap, 0, 150, 25, 'r', 'GR', 10, 1.029)
    xaxis(ax_sp_wrap, -150, 150, 50, 'g', 'SP', 10, 1.051)

    xaxis(ax[1], 0, 200, 0, 'b', 'Rs', 10, 1.003)
    xaxis(ax_rm_wrap, 0.2, 200, 25, 'r', 'Rm', 10, 1.029)
    xaxis(ax_rd_wrap, 0.2, 200, 50, 'g', 'Rd', 10, 1.051)

    xaxis(ax[2], 0.95, 2.95, 0, 'b', 'RHOB', 10, 1.003)
    xaxis(ax_nphi_wrap, -0.15, 1.05, 25, 'r', 'NPHI', 10, 1.029)

    xaxis(ax[3], 40, 240, 0, 'b', 'Sonic', 10, 1.003)

In [ ]:
log_plot(well_train[well_train['WELL'] == '15/9-13'])

In [ ]:
def log_plot_image(logs,plotname,txtname,i,patch_height):
    _, ax = plt.subplots(1,19, figsize = (20, 10), sharey = True, gridspec_kw = {'wspace':0, 'hspace':0})
    ax[0].invert_yaxis()
    ax[1].invert_yaxis()
    ax[2].invert_yaxis()
    ax[3].invert_yaxis()
    ax[4].invert_yaxis()
    ax[5].invert_yaxis()
    ax[6].invert_yaxis()
    ax[7].invert_yaxis()
    ax[8].invert_yaxis()
    ax[9].invert_yaxis()
    ax[10].invert_yaxis()
    ax[11].invert_yaxis()
    ax[12].invert_yaxis()
    ax[13].invert_yaxis()
    ax[14].invert_yaxis()
    ax[15].invert_yaxis()
    ax[16].invert_yaxis()
    ax[17].invert_yaxis()
    ax[18].invert_yaxis()


    ax[0].plot(logs.CALI[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[0].set_xlim(6, 24)
    ax[0].axis('off')
    ax[1].plot(logs.GR[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[1].set_xlim(0, 150)
    ax[1].axis('off')
    ax[2].plot(logs.SP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[2].set_xlim(-150, 150)
    ax[2].axis('off')
    ax[3].plot(logs.SGR[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[3].set_xlim(0, 150)
    ax[3].axis('off')
    ax[4].semilogx(logs.RSHA[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[4].set_xlim(2, 200)
    ax[4].axis('off')
    ax[5].semilogx(logs.RMED[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[5].set_xlim(2, 200)
    ax[5].axis('off')
    ax[6].semilogx(logs.RDEP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[6].set_xlim(2, 200)
    ax[6].axis('off')
    ax[7].semilogx(logs.RXO[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[7].set_xlim(2, 200)
    ax[7].axis('off')
    ax[8].semilogx(logs.RMIC[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[8].set_xlim(2, 200)
    ax[8].axis('off')
    ax[9].plot(logs.NPHI[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[9].set_xlim(-0.15, 1.05)
    ax[9].invert_xaxis()
    ax[9].axis('off')
    ax[10].plot(logs.RHOB[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[10].set_xlim(0.95, 2.95)
    ax[10].invert_xaxis()
    ax[10].axis('off')
    ax[11].plot(logs.PEF[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[11].set_xlim(0, 10)
    ax[11].axis('off')
    ax[12].plot(logs.ROP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[12].set_xlim(0, 50)
    ax[12].axis('off')
    ax[13].plot(logs.ROPA[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[13].set_xlim(0, 50)
    ax[13].axis('off')
    ax[14].plot(logs.DRHO[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[14].set_xlim(-0.2, 1)
    ax[14].axis('off')
    ax[15].plot(logs.DTC[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[15].set_xlim(40, 240)
    ax[15].invert_xaxis()
    ax[15].axis('off')
    ax[16].plot(logs.DTS[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[16].set_xlim(40, 240)
    ax[16].invert_xaxis()
    ax[16].axis('off')
    ax[17].plot(logs.MUDWEIGHT[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[17].set_xlim(0, 150)
    ax[17].axis('off')
    ax[18].plot(logs.BS[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[18].set_xlim(6, 24)
    ax[18].axis('off')
    
     
    plt.savefig(plotname,bbox_inches ="tight",transparent = False)
    #im = Image.open(plotname)
    #newsize = (800, 360)
    #im = im.resize(newsize)
    #im =im.save(plotname)

    with open(txtname, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
    # write a row to the csv file
        writer.writerow(logs.GROUP[i:i+patch_height][:-1].T)
        f.close()

In [ ]:

def randon_list_generator():
    randomlist = []
    for i in range(0,50):
        n = random.randint(0,18)
        if n not in randomlist:
            randomlist.append(n)
    if len(randomlist) == 19: 
        return randomlist
    else:
        return randon_list_generator()
    

randomlist = randon_list_generator()
print(randomlist)


In [ ]:
def log_plot_image_random(logs,plotname,txtname,i,patch_height,randomlist):
    _, ax = plt.subplots(1,19, figsize = (20, 10), sharey = True, gridspec_kw = {'wspace':0, 'hspace':0})
    j=0
    for i in randomlist:
        op= log_plots[i]
        op(well_train[well_train['WELL'] == well_train_names[0]],ax[j],0,700)
        j=j+1
    plt.savefig(plotname,bbox_inches ="tight",transparent = False)
    #im = Image.open(plotname)
    #newsize = (800, 360)
    #im = im.resize(newsize)
    #im =im.save(plotname)

    with open(txtname, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        # write a row to the csv file
        writer.writerow(logs.GROUP[i:i+patch_height])
        f.close()

In [ ]:
def log_plot_image_invert(logs,plotname,txtname,i,patch_height):
    _, ax = plt.subplots(1,19, figsize = (20, 10), sharey = True, gridspec_kw = {'wspace':0, 'hspace':0})



    ax[0].plot(logs.CALI[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[0].set_xlim(6, 24)
    ax[0].axis('off')
    ax[1].plot(logs.GR[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[1].set_xlim(0, 150)
    ax[1].axis('off')
    ax[2].plot(logs.SP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[2].set_xlim(-150, 150)
    ax[2].axis('off')
    ax[3].plot(logs.SGR[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[3].set_xlim(0, 150)
    ax[3].axis('off')
    ax[4].semilogx(logs.RSHA[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[4].set_xlim(2, 200)
    ax[4].axis('off')
    ax[5].semilogx(logs.RMED[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[5].set_xlim(2, 200)
    ax[5].axis('off')
    ax[6].semilogx(logs.RDEP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[6].set_xlim(2, 200)
    ax[6].axis('off')
    ax[7].semilogx(logs.RXO[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[7].set_xlim(2, 200)
    ax[7].axis('off')
    ax[8].semilogx(logs.RMIC[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[8].set_xlim(2, 200)
    ax[8].axis('off')
    ax[9].plot(logs.NPHI[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[9].set_xlim(-0.15, 1.05)
    ax[9].invert_xaxis()
    ax[9].axis('off')
    ax[10].plot(logs.RHOB[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[10].set_xlim(0.95, 2.95)
    ax[10].invert_xaxis()
    ax[10].axis('off')
    ax[11].plot(logs.PEF[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[11].set_xlim(0, 10)
    ax[11].axis('off')
    ax[12].plot(logs.ROP[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[12].set_xlim(0, 50)
    ax[12].axis('off')
    ax[13].plot(logs.ROPA[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[13].set_xlim(0, 50)
    ax[13].axis('off')
    ax[14].plot(logs.DRHO[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[14].set_xlim(-0.2, 1)
    ax[14].axis('off')
    ax[15].plot(logs.DTC[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[15].set_xlim(40, 240)
    ax[15].invert_xaxis()
    ax[15].axis('off')
    ax[16].plot(logs.DTS[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[16].set_xlim(40, 240)
    ax[16].invert_xaxis()
    ax[16].axis('off')
    ax[17].plot(logs.MUDWEIGHT[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[17].set_xlim(0, 150)
    ax[17].axis('off')
    ax[18].plot(logs.BS[i:i+patch_height], list(range(i,i+patch_height)), 'b')
    ax[18].set_xlim(6, 24)
    ax[18].axis('off')
    
     
    plt.savefig(plotname,bbox_inches ="tight",transparent = False)
    #im = Image.open(plotname)
    #newsize = (800, 360)
    #im = im.resize(newsize)
    #im =im.save(plotname)

    with open(txtname, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
    # write a row to the csv file
        writer.writerow(logs.GROUP[i:i+patch_height])
        f.close()

In [ ]:
count =0
for j in range(start_well,end_well,1):
        well_shape = len(well_train[well_train['WELL'] == well_train_names[j]])
        
        for i in tqdm(range(0, well_shape, 350)):
                plotname = "well-images/well_"+str(j)+"_"+str(i)+".jpg"
                txtname = "well-images/well_"+str(j)+"_"+str(i)+".csv"
                
                if i+700 > well_shape:
                        well_not700 = well_shape-i
                        #print(i, well_not700+i ,i-(700-well_not700), i-(700-well_not700)+700)
                        log_plot_image(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i-(700-well_not700),700)
                else:
                        log_plot_image(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i,700)
                
                
                        
                count = count+1
print(count)

In [ ]:
count =0
for j in range(start_well,end_well,1):
        well_shape = len(well_train[well_train['WELL'] == well_train_names[j]])
        
        for i in tqdm(range(0, well_shape, 350)):
                plotname = "well-images/random_well_"+str(j)+"_"+str(i)+".jpg"
                txtname = "well-images/random_well_"+str(j)+"_"+str(i)+".csv"
                randomlist = randon_list_generator()
                if i+700 > well_shape:
                        well_not700 = well_shape-i
                        #print(i, well_not700+i ,i-(700-well_not700), i-(700-well_not700)+700)
                        log_plot_image_random(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i-(700-well_not700),700, randomlist)
                else:
                        log_plot_image_random(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i,700, randomlist)
                
                
                        
                count = count+1
print(count)

In [ ]:
count =0
for j in range(start_well,end_well,1):
        well_shape = len(well_train[well_train['WELL'] == well_train_names[j]])
        
        for i in tqdm(range(0, well_shape, 350)):
                plotname = "well-images/invert_well_"+str(j)+"_"+str(i)+".jpg"
                txtname = "well-images/invert_well_"+str(j)+"_"+str(i)+".csv"
                
                if i+700 > well_shape:
                        well_not700 = well_shape-i
                        #print(i, well_not700+i ,i-(700-well_not700), i-(700-well_not700)+700)
                        log_plot_image_invert(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i-(700-well_not700),700)
                else:
                        log_plot_image_invert(well_train[well_train['WELL'] == well_train_names[0]],plotname,txtname,i,700)
                
                
                        
                count = count+1
print(count)